<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Request-Kiwi.com" data-toc-modified-id="Request-Kiwi.com-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Request Kiwi.com</a></span><ul class="toc-item"><li><span><a href="#Get-Data-and-loads" data-toc-modified-id="Get-Data-and-loads-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get Data and loads</a></span></li><li><span><a href="#Convert-json-to-dataframe" data-toc-modified-id="Convert-json-to-dataframe-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Convert json to dataframe</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Flight-to-home" data-toc-modified-id="Flight-to-home-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Flight to home</a></span></li></ul></li></ul></div>

In [260]:
import requests
import json

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

### Request Kiwi.com

- From : Bristol, UK (airport: BRS (IATA)) <br>
- To : Prague, CZ (airport: PRG (IATA)) <br>
- When : within 5 days before 12.24 (between 12.19 - 12.23) <br>
- Money : Less, better
- Vehcile : aircraft (Options : train, bus)

In [259]:
"""
Initialization
"""
kiwi_endpoint = 'https://api.skypicker.com'
kiwi_basic_endpoint = 'https://api.skypicker.com/flights'
# kiwi_normal_endpoint = 'https://api.skypicker.com/traveling_salesman'

data_para_flight = {
    "adults": 1,
    "partner" : "picky",
    "date_from" : "19/12/2019",
    "date_to" : "23/12/2019",
    "fly_type" : "oneway",
    "fly_from" : "BRS",
    "fly_to" : "PRG",
    "vehicle_type" : "aircraft",
    "sort" : "price",
    "max_stopovers" : 0
}

# data_para_train = {
#     "adults": 1,
#     "partner" : "picky",
#     "date_from" : "19/12/2019",
#     "date_to" : "23/12/2019",
#     "fly_type" : "oneway",
#     "from" : "B",
#     "to" : "LON",
#     "vehicle_type" : "train",
#     "sort" : "price",
#     "max_stopovers" : 2
# }
print(data_para_flight)
# data = requests.get(url = kiwi_basic_endpoint, params= data_para)

{'adults': 1, 'partner': 'picky', 'date_from': '19/12/2019', 'date_to': '23/12/2019', 'fly_type': 'oneway', 'fly_from': 'BRS', 'fly_to': 'PRG', 'vehicle_type': 'aircraft', 'sort': 'price', 'max_stopovers': 0}


#### Get Data and loads

In [251]:
data_flight = requests.get(url = kiwi_basic_endpoint, params= data_para_flight)
json_data_flight = json.loads(data_flight.content)

In [252]:
# data_train = requests.get(url = kiwi_basic_endpoint, params= data_para_train)
# json_data_train = json.loads(data_train.content)
# data_train

#### Convert json to dataframe

In [253]:
df_data_flight = json_normalize(json_data_flight['data'])

In [254]:
df_data_flight.shape

(5, 50)

#### Preprocessing 

In [255]:
df_data_flight.columns

Index(['aTime', 'aTimeUTC', 'airlines', 'baglimit.hand_height',
       'baglimit.hand_length', 'baglimit.hand_weight', 'baglimit.hand_width',
       'baglimit.hold_dimensions_sum', 'baglimit.hold_height',
       'baglimit.hold_length', 'baglimit.hold_weight', 'baglimit.hold_width',
       'bags_price.1', 'bags_price.2', 'booking_token', 'cityFrom', 'cityTo',
       'conversion.EUR', 'countryFrom.code', 'countryFrom.name',
       'countryTo.code', 'countryTo.name', 'dTime', 'dTimeUTC', 'deep_link',
       'distance', 'duration.departure', 'duration.return', 'duration.total',
       'facilitated_booking_available', 'flyFrom', 'flyTo', 'fly_duration',
       'found_on', 'has_airport_change', 'id', 'mapIdfrom', 'mapIdto',
       'nightsInDest', 'p1', 'p2', 'p3', 'pnr_count', 'price', 'quality',
       'route', 'routes', 'transfers', 'type_flights', 'virtual_interlining'],
      dtype='object')

In [256]:
cols = np.array(['airlines', 'cityFrom', 'cityTo',
                'conversion.EUR', 'dTimeUTC', 'duration.departure', 'duration.total', 
                 'price', 'quality', 'routes', 'type_flights'])
df_flight = pd.DataFrame(df_data_flight[cols])
# df_flight

In [257]:
# Convert departure time (seconds) to datetime
df_flight['date_time'] = pd.to_datetime(df_flight['dTimeUTC'], unit= 's')
# Convert duration (seconds) to hours
df_flight['duration_h'] = round(df_flight['duration.departure'] / 3600, 2)

#### Flight to home

In [258]:
df_flight

,airlines,cityFrom,cityTo,conversion.EUR,dTimeUTC,duration.departure,duration.total,price,quality,routes,type_flights,date_time,duration_h
0,[U2],Bristol,Prague,102,1577107200,7200,7200,102,129.99993,"[[BRS, PRG]]",[lcc-U2],2019-12-23 13:20:00,2.0
1,[U2],Bristol,Prague,108,1576761600,7200,7200,108,135.99993,"[[BRS, PRG]]",[lcc-U2],2019-12-19 13:20:00,2.0
2,[U2],Bristol,Prague,118,1576848000,7200,7200,118,145.99993,"[[BRS, PRG]]",[lcc-U2],2019-12-20 13:20:00,2.0
3,[U2],Bristol,Prague,123,1577034900,7200,7200,123,150.99993,"[[BRS, PRG]]",[lcc-U2],2019-12-22 17:15:00,2.0
4,[U2],Bristol,Prague,132,1576911900,7200,7200,132,159.99993,"[[BRS, PRG]]",[lcc-U2],2019-12-21 07:05:00,2.0


The ideal direct flight should be the one charges only 102 euro, departures on 2019-12-23 at 13:20:00, from Bristol to Prague.